<a href="https://colab.research.google.com/github/pushkar3422/testdemo/blob/master/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text


In [47]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam (1).csv


In [59]:
import pandas as pd
import io
  
df = pd.read_csv(io.StringIO(uploaded['spam.csv'].decode('latin-1')))
df.head()


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [60]:
df.groupby('Category').describe()

Message                                                               
           count unique                                                top freq
Category                                                                       
ham         4825   4516                             Sorry, I'll call later   30
spam         747    653  Please call our customer service representativ...    4

In [62]:
df['spam'] = df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Message'], df['spam'], test_size=0.2, stratify=df['spam'])

In [82]:
len(X_train )

4457

In [83]:
len(X_test )

1115

In [84]:
bert_preprocess=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')


In [85]:
def get_sentence_embeding(sentence):
  preprocessed_text =bert_preprocess(sentence)
  return bert_encoder(preprocessed_text)['pooled_output']

In [86]:
e = get_sentence_embeding([
                           
                           "banana",
                           "grapes",
                           "mango",
                           "jeff bezos",
                           "elon musk"
                           
                           
                           ])

In [87]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[1]],[e[0]])


array([[0.99110883]], dtype=float32)

In [88]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.3, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs=[l])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [89]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [76]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/5
  6/131 [>.............................] - ETA: 30:57 - loss: 0.5788 - accuracy: 0.7344

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
reviews = [
           'FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv',
'Even my brother is not like to speak with me. They treat me like aids patent.',
'As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune',
'WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.',
'Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,


]
model.predict(reviews)